In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import hvplot.pandas

In [ ]:
fn = "../subworkflows/pypsa-earth-sec/data/industrial_database.csv"

In [ ]:
OUTPUT = "../../results/graphics_general/morocco_industry.pdf"

In [ ]:
idb = pd.read_csv(fn, index_col=0)

In [ ]:
# Filter for country MA
idb = idb[idb.index == "MA"]

In [ ]:
# Rescale unit of HVC from bpd to kt/yr ("One BPD also becomes 49.8 tonnes per year.[14]"https://en.wikipedia.org/wiki/Barrel_(unit)#cite_note-statrev-15)
idb.loc[idb["unit"] == "bpd", "capacity"] *= 49.8 / 1000 # from bpd to kt/yr
# rename bpd to kt/yr
idb.loc[idb["unit"] == "bpd", "unit"] = "kt/yr"

In [ ]:
# Rename HVC to Refinery
idb.loc[idb["technology"] == "HVC", "technology"] = "Refinery"
# Rename Electric arc to Steel
idb.loc[idb["technology"] == "Electric arc", "technology"] = "Steel"

In [ ]:
idb.head()

In [ ]:
px.scatter_mapbox(
    idb,
    lat="y",
    lon="x",
    mapbox_style="carto-positron",
    color="technology",
    #size="capacity",
    # size
    #hover_name="name",
    zoom=4,
    height=600,
)

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import cartopy.crs as ccrs
import cartopy

HV plot

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
colors = {
    "Cement": "grey",
    "Electric arc": "red",
    "Steel": "red",
    "HVC": "purple",
    "Refinery": "black",
}

In [ ]:
df = gpd.GeoDataFrame(idb, geometry=gpd.points_from_xy(idb["x"], idb["y"]), crs="EPSG:4326")
crs = ccrs.AlbersEqualArea()

#df = df.cx[-12:40, 35:72]
df = df.to_crs(crs.proj4_init)

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={"projection": crs})

ax.add_feature(cartopy.feature.COASTLINE.with_scale("10m"), linewidth=0.2, zorder=2)
ax.add_feature(cartopy.feature.BORDERS.with_scale("10m"), linewidth=0.2, zorder=2)
ax.add_feature(cartopy.feature.OCEAN, facecolor="white", zorder=1)
ax.add_feature(cartopy.feature.LAND, edgecolor="black", facecolor="#f1f2f2", zorder=1) #d1d1d1

df.plot(
    ax=ax,
    column="technology",
    markersize=df["capacity"] / 15,
    alpha=0.7,
    legend=True,
    cmap=mcolors.ListedColormap(
        pd.Series(df.technology.unique()).sort_values().map(colors).values
    ),
    legend_kwds=dict(title="Technology (size ~ capacity)", frameon=True, facecolor="white", loc="upper left"),
)
#cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, zoom=10)

minx, miny, maxx, maxy = df.total_bounds
ax.set_xlim(minx*1.1, maxx*0.5)
ax.set_ylim(miny*0.95, maxy*1.05)

ax.set_frame_on(False)
ax.set_facecolor("white")

plt.savefig(OUTPUT, bbox_inches="tight")